# Start with data and package importing 


In [4293]:
import os
import pandas as pd
import datetime as dt
import re
from pathlib import Path

## Actual dates of Super Bowl
Cleaned up this dataset primarily to get the dates in datetime format. 
Also has some locations which might be useful but maybe not too. 
Text file scraped from https://www.pro-football-reference.com/super-bowl/

In [4294]:
df3 = pd.read_csv('..\\Data\\superbowls.txt', sep='\t', lineterminator='\r')
# Text file scraped from https://www.pro-football-reference.com/super-bowl/
## prime use here is for the superbowl dates and locations
## get dates
df3["Date"] = df3.Date.str.replace("\n", "")
df3["Date"] = df3.Date.str.replace("Feb", "02")
df3["Date"] = df3.Date.str.replace(",", "")
days = df3.Date.str.split(expand=True)[1].str.zfill(2)
df3["Year"] = df3.Date.str.split(expand=True)[2]
df3["Date"] = df3.Date.str.split(expand=True)[0] + "-" + days + "-" + df3.Date.str.split(expand=True)[2]
df3["Date"] = pd.to_datetime(df3["Date"])

print(df3.head())
df3.info()

        Date         SB                Winner Pts                Loser Pts.1  \
0 2023-02-12  LVII (57)    Kansas City Chiefs  38  Philadelphia Eagles    35   
1 2022-02-13   LVI (56)      Los Angeles Rams  23   Cincinnati Bengals    20   
2 2021-02-07    LV (55)  Tampa Bay Buccaneers  31   Kansas City Chiefs     9   
3 2020-02-02   LIV (54)    Kansas City Chiefs  31  San Francisco 49ers    20   
4 2019-02-03  LIII (53)  New England Patriots  13     Los Angeles Rams     3   

               MVP                Stadium           City       State  Year  
0  Patrick Mahomes     State Farm Stadium       Glendale     Arizona  2023  
1      Cooper Kupp           SoFi Stadium      Inglewood  California  2022  
2        Tom Brady  Raymond James Stadium          Tampa     Florida  2021  
3  Patrick Mahomes      Hard Rock Stadium  Miami Gardens     Florida  2020  
4   Julian Edelman  Mercedes-Benz Stadium        Atlanta     Georgia  2019  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entr

Drop the rows without dates.

In [4295]:
df3 = df3.drop(index=[35,47])

## Main set from kaggle.
This needed some cleaning just to load due to poor data entry in the csv 

https://www.kaggle.com/datasets/prondeau/superbowlads

In [4296]:
df4 = pd.read_csv('..\\Data\\superbowl-ads.csv')
# This was cleaned up from Kaggle https://www.kaggle.com/datasets/prondeau/superbowlads
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563 entries, 0 to 562
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Year           563 non-null    object
 1   Product Type   563 non-null    object
 2   Product/Title  562 non-null    object
 3   Plot/Notes     547 non-null    object
dtypes: object(4)
memory usage: 17.7+ KB


Look at the shape and first few rows.

In [4297]:
print(df4.shape)
df4.head()

(563, 4)


,Year,Product Type,Product/Title,Plot/Notes
0,1969,Car,"Chrysler ""Scuba Diver""[4]",A scuba diver in a dreamy sequence with groovy...
1,1970,Car,"Pontiac""Humbler""",A young man pulled up in his GTO to a drive-in...
2,1975,Security,"Master Lock""Rifle""",A man attempts to break a Master Lock with a r...
3,1976,Copy,"Xerox""Monks""",A monk must make 500 handwritten scriptures fo...
4,1979,Soft drink,"Coca-Cola""Hey Kid, Catch!""","A young kid tries to talk to ""Mean"" Joe Greene..."


The file has a listing of 563 superbowl commercials with the year, product type, product/commmercial title, and sometimes a plot.

We are mostly interested in the product type and getting the parent company from the product.

Look at the year first.

In [4298]:
df4.Year.unique()
## Something wrong here with year 

array(['1969', '1970', '1975', '1976', '1979', '1981', '1982', '1984',
       '1988', '1989', '1990', '1992', '1993', '1994', '1996', '1997',
       '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', 'Food, TV series'],
      dtype=object)

Clearly an issue here. Lets see the specific problem.

In [4299]:
df4[df4.Year == 'Food, TV series']
## looks like year should be 2020 and shift the two others over for this entry
## lets make better column names first

,Year,Product Type,Product/Title,Plot/Notes
561,"Food, TV series","Pringles, Rick and Morty",NaN,After Morty enthusiastically enters the living...


The year can be determined to be 2020 and the other columns shifted into place.

Next, let's get better names for the columns and fix the row in question.

In [4300]:
df4 = df4.rename(columns={"Product Type":'Product_type', 'Product/Title':'Brand_title','Plot/Notes':'Plot'})

In [4301]:
df4.iloc[561] = df4.iloc[561].shift(1)
df4.iloc[561].Year = '2020'
df4.iloc[561]

Year                                2020
Product_type             Food, TV series
Brand_title     Pringles, Rick and Morty
Plot                                 NaN
Name: 561, dtype: object

# Now add the info from `df3` by year.

In [4302]:
df4 = df4.merge(df3[['Year','Date']] , on='Year', how='left') 
df4.Date = pd.to_datetime(df4.Date, utc=True)
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 563 entries, 0 to 562
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   Year          563 non-null    object             
 1   Product_type  563 non-null    object             
 2   Brand_title   563 non-null    object             
 3   Plot          546 non-null    object             
 4   Date          563 non-null    datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), object(4)
memory usage: 26.4+ KB


Probably won't want most of the columns but will leave them for now.

Now, split the `Brand_title` into two columns  one for `Brand` and one for the commercial `title`.

In [4303]:
df4[['Brand','Title']] = df4['Brand_title'].str.split('"',expand=True)[[0,1]]
df4.head()

,Year,Product_type,Brand_title,Plot,Date,Brand,Title
0,1969,Car,"Chrysler ""Scuba Diver""[4]",A scuba diver in a dreamy sequence with groovy...,1969-01-12 00:00:00+00:00,Chrysler,Scuba Diver
1,1970,Car,"Pontiac""Humbler""",A young man pulled up in his GTO to a drive-in...,1970-01-11 00:00:00+00:00,Pontiac,Humbler
2,1975,Security,"Master Lock""Rifle""",A man attempts to break a Master Lock with a r...,1975-01-12 00:00:00+00:00,Master Lock,Rifle
3,1976,Copy,"Xerox""Monks""",A monk must make 500 handwritten scriptures fo...,1976-01-18 00:00:00+00:00,Xerox,Monks
4,1979,Soft drink,"Coca-Cola""Hey Kid, Catch!""","A young kid tries to talk to ""Mean"" Joe Greene...",1979-01-21 00:00:00+00:00,Coca-Cola,"Hey Kid, Catch!"


Look at the different `Product_type`

In [4304]:
product_type_orig = df4.Product_type.unique()
df4.Product_type.unique()

array(['Car', 'Security', 'Copy', 'Soft drink', 'Camera', 'Computer',
       'Restaurant', 'Beer', 'Food', 'Credit card', 'Soft Drink',
       'Footwear', 'Clothing', 'Film', 'Mail', 'Shoes', 'Sports', 'Tire',
       'Website', 'Candy', 'TV', 'Gaming', 'Potato chips', 'PSA',
       'Technology', 'Manufacturing', 'Mobile phone', 'TV show',
       'Video Game', 'Store', 'Truck', 'Product type', 'Car accessories',
       'Adhesives', 'Antifungal medication', 'Feminine care products',
       'Insurance', 'Lingerie', 'Phone', 'Phone accessory', 'Service',
       'Skincare products', 'Software', 'Game', 'Toothpaste',
       'Building materials', 'Cleaner', 'Tax preparation', 'TV series',
       'Video games', 'Web hosting', 'Wireless', 'Airlines', 'Car / Film',
       'Drink', 'Food / Drink', 'Headphones', 'Investments',
       'Laundry detergent', 'Loans', 'Retail', 'TV special', 'Travel',
       'Video game', 'Alcohol', 'Alcohol/TV series', 'Beverage',
       'Smart speaker', 'NFL', 'Video

Let's combine some types. (but save the original names as `product_type_orig`)


In [4305]:
df4['Product_type'] = df4['Product_type'].str.replace("Truck","Car").str.replace("Soft Drink", "Soft drink").str.replace("TV special", "TV").str.replace("TV series", "TV").str.replace("TV show", "TV")
df4['Product_type'] = df4['Product_type'].str.replace("Potato chips","Food").str.replace("Drink","Alcohol").str.replace("Beer","Alcohol").str.replace("Candy", "Food")
df4['Product_type'] = df4['Product_type'].str.replace("Copy","Technology").str.replace("Camera","Technology").str.replace("Computer","Technology")
df4['Product_type'] = df4['Product_type'].str.replace("Footwear","Shoes").str.replace("Lingerie","Clothing").str.replace("Food, TV series", "Food")
df4['Product_type'] = df4['Product_type'].str.replace("Phone","Wireless").str.replace("Mobile phone","Wireless").str.replace("TV special", "TV")
df4['Product_type'] = df4['Product_type'].str.replace("Restaurant","Food").str.replace("Shoes","Clothing").str.replace("Mail", "Service")
df4 = df4.drop(df4[df4.Product_type == 'Product type'].index)
df4['Product_type'] = df4['Product_type'].str.replace("Security","Goods").str.replace("Tire", "Goods").str.replace("Laundry detergent","Goods").str.replace("Cleaner","Goods")
df4['Product_type'] = df4['Product_type'].str.replace("Building materials","Goods").str.replace("Antifungal medication", "Goods").str.replace("Skincare products","Goods").str.replace("Cleaner","Goods")
df4['Product_type'] = df4['Product_type'].str.replace("Toothpaste","Goods").str.replace("Adhesives", "Goods").str.replace("Feminine care products","Goods").str.replace("Cleaner","Goods")
df4['Product_type'] = df4['Product_type'].str.replace("Credit card","Service").str.replace("Insurance","Service").str.replace("Loans","Service").str.replace("Tax preparation","Service")
df4['Product_type'] = df4['Product_type'].str.replace("Airlines","Service").str.replace("Investments","Service").str.replace("Travel","Service").str.replace("Web hosting","Service")
df4['Product_type'] = df4['Product_type'].str.replace("Video gaming","Goods").str.replace("Video games","Goods").str.replace("Video game","Goods").str.replace("Video Game","Goods")
df4['Product_type'] = df4['Product_type'].str.replace("Food / Alcohol","Food").str.replace("Food, TV","Food").str.replace("Alcohol/TV","Alcohol")
df4['Product_type'] = df4['Product_type'].str.replace("NFL","Sports").str.replace("Software","Service").str.replace("Smart speaker","Service").str.replace("Retail","Service")
df4['Product_type'] = df4['Product_type'].str.replace("Headphones","Goods").str.replace("Wireless accessory","Goods").str.replace("Game","Goods").str.replace("Beverage","Soft drink")
df4['Product_type'] = df4['Product_type'].str.replace("Manufacturing","Goods").str.replace("Car accessories","Goods").str.replace("Store","Goods").str.replace("Beverage","Soft drink")
df4 = df4.drop(df4[df4.Product_type == 'PSA'].index)
df4[df4.Product_type == 'Services']


,Year,Product_type,Brand_title,Plot,Date,Brand,Title


In [4306]:
df4.groupby('Product_type').count().sort_values('Brand', ascending=False)

,Year,Brand_title,Plot,Date,Brand,Title
Product_type,,,,,,
Film,140,140,135,140,140,3
Car,81,81,80,81,81,80
Alcohol,60,60,58,60,60,60
Food,58,58,54,58,58,55
Website,45,45,44,45,45,44
Goods,33,33,32,33,33,31
Service,32,32,32,32,32,30
Soft drink,30,30,29,30,30,30
TV,23,23,23,23,23,3


This still may be too many but there are already some major groups `Film`, `Car`, `Alcohol` and `Food` are about 60% of the commercials in the dataset. Regardless, save as `product_type_final` 

In [4307]:
product_type_final = df4.Product_type.unique()
product_type_final 

array(['Car', 'Goods', 'Technology', 'Soft drink', 'Food', 'Alcohol',
       'Service', 'Clothing', 'Film', 'Sports', 'Website', 'TV', 'Gaming',
       'Wireless', 'Car / Film'], dtype=object)

## films is the biggest so let's see what we can do with those.

In [4308]:
films = df4[df4.Product_type == 'Film']
films.head()


,Year,Product_type,Brand_title,Plot,Date,Brand,Title
27,1996,Film,Independence Day,TV promo.,1996-01-28 00:00:00+00:00,Independence Day,None
30,1997,Film,Dante's Peak,TV promo.,1997-01-26 00:00:00+00:00,Dante's Peak,None
31,1997,Film,The Lost World: Jurassic Park,TV promo.,1997-01-26 00:00:00+00:00,The Lost World: Jurassic Park,None
32,1997,Film,The Saint,TV promo.,1997-01-26 00:00:00+00:00,The Saint,None
43,1998,Film,Armageddon,TV promo.,1998-01-25 00:00:00+00:00,Armageddon,None


First, I used ChatGPT to get the studio that made each movie. The result was copied to Sublime Text keeping the original index from `df4` as `row` when loaded into the df `filmstudios`.

In [4309]:
filmstudios = pd.read_csv('..\\Data\\films.txt', sep='-', lineterminator='\r', header=None)
filmstudios = filmstudios.rename(columns={0:'row',1:'Studio'})
filmstudios.head()

,row,Studio
0,27,20th Century Fox
1,30,Universal Pictures
2,31,Universal Pictures
3,32,Paramount Pictures
4,43,Touchstone Pictures


Now, got the Ticket symbols by using ChatGPT again on `filmstudios` saved in Sublime Text as `filmticker`

In [4310]:
film_ticker = pd.read_csv('..\\Data\\filmticker.txt', sep='-', lineterminator='\r', header=None)
film_ticker =film_ticker.rename(columns={0:'row',1:'Ticker'})

In [4311]:
film_ticker = film_ticker.drop(film_ticker.index[[140,141]])
film_ticker.tail()

,row,Ticker
135,\n528 2019 Monkeypaw Productions,CMCSA
136,\n529 2019 Nickelodeon Movies,VIAC
137,\n555 2020 Universal Pictures,CMCSA
138,\n556 2020 Universal Pictures,CMCSA
139,\n557 2020 Walt Disney Pictures,DIS


In [4312]:
film_ticker['row'] = film_ticker['row'].str.split(' ',expand=True)[0]
film_ticker['row'] = film_ticker['row'].str.replace("\n", "")
film_ticker['row'][[103,104]] = [334, 335] 
film_ticker['row'][[103,104]]
film_ticker['row'] = pd.to_numeric(film_ticker['row'], downcast='integer')

Finished cleaning up now merge with films to add the ticker symbol we could probably add in the studio but not much need for that.

Merge with `films`

In [4313]:
films = films.merge(film_ticker, how='left', left_index=True, right_on='row')
films.head()

,Year,Product_type,Brand_title,Plot,Date,Brand,Title,row,Ticker
0,1996,Film,Independence Day,TV promo.,1996-01-28 00:00:00+00:00,Independence Day,None,27,FOX
1,1997,Film,Dante's Peak,TV promo.,1997-01-26 00:00:00+00:00,Dante's Peak,None,30,CMCSA
2,1997,Film,The Lost World: Jurassic Park,TV promo.,1997-01-26 00:00:00+00:00,The Lost World: Jurassic Park,None,31,CMCSA
3,1997,Film,The Saint,TV promo.,1997-01-26 00:00:00+00:00,The Saint,None,32,VIAC
4,1998,Film,Armageddon,TV promo.,1998-01-25 00:00:00+00:00,Armageddon,None,43,DIS


In [4314]:
films = films.set_index("row")
#films.head()

## TV shows next

In [4315]:
tvshows = df4[df4.Product_type == 'TV']
#tvshows[['Year','Brand']]

## Use this on ChatGPT to get the tickers again

In [4316]:
tv_ticker = pd.read_csv('..\\Data\\tv_ticker.txt', sep=' ', lineterminator='\r', header=None)
tv_ticker = tv_ticker.rename(columns={0:'row',3:'Ticker', 1: 'Year', 2: 'Network'})
tv_ticker = tv_ticker.drop(columns=['Year', 'Network'])
tv_ticker['row'] = pd.to_numeric(tv_ticker['row'], downcast='integer')
tvshows = tvshows.merge(tv_ticker, how='left', left_index=True, right_on='row')
tvshows = tvshows.set_index("row")
tvshows.head()

,Year,Product_type,Brand_title,Plot,Date,Brand,Title,Ticker
row,,,,,,,,
95,2000,TV,"Oxygen""Baby Girl Power""","At a maternity ward, a pink cap for a baby gir...",2000-01-30 00:00:00+00:00,Oxygen,Baby Girl Power,OXYGN
252,2010,TV,Late Show with David Letterman,"David Letterman, Oprah Winfrey, and Jay Leno a...",2010-02-07 00:00:00+00:00,Late Show with David Letterman,None,CBS
253,2010,TV,NFL Full Contact,"On Groundhog Day, the mayor pulls out a small ...",2010-02-07 00:00:00+00:00,NFL Full Contact,None,NFLX
413,2017,TV,24: Legacy (Fox) N/A,Promo[98],2017-02-05 00:00:00+00:00,24: Legacy (Fox) N/A,None,FOXA
414,2017,TV,Empire (Fox) N/A,Promo[98],2017-02-05 00:00:00+00:00,Empire (Fox) N/A,None,FOXA


## Looks good  move on to the next group `Car`

In [4317]:
cars = df4[df4.Product_type == 'Car']
#cars[['Year','Brand']][41:]

Use ChatGPT again to generate txt file `car_ticker`

In [4318]:
car_ticker = pd.read_csv('..\\Data\\car_ticker.txt', sep=' ', lineterminator='\r', header=None)
car_ticker = car_ticker.rename(columns={0:'row',3:'Ticker', 1: 'Year', 2: 'Car'})
car_ticker = car_ticker.drop(columns=['Year', 'Car'])
car_ticker['row'] = pd.to_numeric(car_ticker['row'], downcast='integer')
cars = cars.merge(car_ticker, how='left', left_index=True, right_on='row')
cars = cars.set_index("row")
#cars.head()

## Now `Alcohol`

In [4319]:
alcohol = df4[df4.Product_type == 'Alcohol']
#alcohol[['Year','Brand']]

In [4320]:
alcohol_ticker = pd.read_csv('..\\Data\\alcohol_ticker.txt', sep=' ', lineterminator='\r', header=None)
alcohol_ticker = alcohol_ticker.rename(columns={0:'row',3:'Ticker', 1: 'Year', 2: 'beer'})
alcohol_ticker = alcohol_ticker.drop(columns=['Year', 'beer'])
alcohol_ticker['row'] = pd.to_numeric(alcohol_ticker['row'], downcast='integer')
alcohol = alcohol.merge(alcohol_ticker, how='left', left_index=True, right_on='row')
alcohol = alcohol.set_index("row")
#alcohol.head()

## Next `Food`

In [4321]:
food = df4[df4.Product_type == 'Food']
#food[['Year','Brand']]

In [4322]:
food_ticker = pd.read_csv('..\\Data\\food_ticker.txt', sep='-', lineterminator='\r', header=None)
food_ticker[0] = food_ticker[0].str.split(' ',expand=True)[0].str.replace('\n','')
food_ticker = food_ticker.rename(columns={0:'row',1:'Ticker'})
food_ticker['row'] = pd.to_numeric(food_ticker['row'])
food = food.merge(food_ticker, how='left', left_index=True, right_on='row')
food= food.set_index("row")
#food.head()

## Next `Website`

In [4323]:
website = df4[df4.Product_type == 'Website']
#website[['Year','Brand']]

In [4324]:
website_ticker = pd.read_csv('..\\Data\\website_ticker.txt', sep='-', lineterminator='\r', header=None)
website_ticker[0] = website_ticker[0].str.split(' ',expand=True)[0].str.replace('\n','')
website_ticker = website_ticker.rename(columns={0:'row',1:'Ticker'})
website_ticker['row'] = pd.to_numeric(website_ticker['row'], downcast='integer')
website = website.merge(website_ticker, how='left', left_index=True, right_on='row')
website = website.set_index("row")
#website.head()

## Next `Goods`

In [4325]:
goods = df4[df4.Product_type == 'Goods']
#goods[['Year','Brand']]

In [4326]:
goods_ticker = pd.read_csv('..\\Data\\goods_ticker.txt', sep='(', lineterminator='\r', header=None)
goods_ticker[0] = goods_ticker[0].str.split(' ',expand=True)[0].str.replace('\n','')
goods_ticker[1]  = goods_ticker[1].str.replace(')', '')
goods_ticker = goods_ticker.rename(columns={0:'row',1:'Ticker'})
goods_ticker['row'] = pd.to_numeric(goods_ticker['row'], downcast='integer')
goods = goods.merge(goods_ticker, how='left', left_index=True, right_on='row')
goods = goods.set_index("row")
goods.head()

C:\Users\micha\AppData\Local\Temp\ipykernel_13364\444714591.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  goods_ticker[1]  = goods_ticker[1].str.replace(')', '')


,Year,Product_type,Brand_title,Plot,Date,Brand,Title,Ticker
row,,,,,,,,
2,1975,Goods,"Master Lock""Rifle""",A man attempts to break a Master Lock with a r...,1975-01-12 00:00:00+00:00,Master Lock,Rifle,NYSE:FBHS
11,1984,Goods,"Master Lock""The Doubters",NaN,1984-01-22 00:00:00+00:00,Master Lock,The Doubters,NYSE:FBHS
58,1998,Goods,"Michelin""Drill""",A man goes up to his car and drills a hole in ...,1998-01-25 00:00:00+00:00,Michelin,Drill,OTC: MGDDF
218,2009,Goods,"Bridgestone""The Potato Heads""",Mr. Potato Head is being nagged by his wife wh...,2009-02-01 00:00:00+00:00,Bridgestone,The Potato Heads,OTC: BRDCY
246,2010,Goods,"Bridgestone""Killer Whale""",Three men have a killer whale in their small v...,2010-02-07 00:00:00+00:00,Bridgestone,Killer Whale,OTC: BRDCY


This time ChatGPT inserted the exchange but the colon should make it easy to remove if needed.

## Now `Service`

In [4327]:
services = df4[df4.Product_type == 'Service']
#services[['Year','Brand']]

In [4328]:
services_ticker = pd.read_csv('..\\Data\\services_ticker.txt', sep='-', lineterminator='\r', header=None)
services_ticker[0] = services_ticker[0].str.split(' ',expand=True)[0].str.replace('\n','')
services_ticker = services_ticker.rename(columns={0:'row', 1:'Ticker'})
services_ticker['row'] = pd.to_numeric(services_ticker['row'])
services = services.merge(services_ticker, how='left', left_index=True, right_on='row')
services = services.set_index("row")
services.head()



,Year,Product_type,Brand_title,Plot,Date,Brand,Title,Ticker
row,,,,,,,,
15,1990,Service,"American Express""The Race""",Paul Newman races a Top-Fuel Dragster on a Min...,1990-01-28 00:00:00+00:00,American Express,The Race,AXP
42,1998,Service,"Visa""Elephant""",A little girl talks about how she would want a...,1998-01-25 00:00:00+00:00,Visa,Elephant,V
52,1998,Service,"FedEx""We Apologize""","A colored ""Please Stand By"" test bar is on the...",1998-01-25 00:00:00+00:00,FedEx,We Apologize,FDX
64,1999,Service,"MasterCard""Toons""",Characters from the Hanna-Barbera cartoons are...,1999-01-31 00:00:00+00:00,MasterCard,Toons,MA
77,1999,Service,"FedEx""Stanley Cup""",An agent is supposed to deliver the Stanley Cu...,1999-01-31 00:00:00+00:00,FedEx,Stanley Cup,FDX


## Next `Soft drink`

In [4329]:
Soft_drink = df4[df4.Product_type == 'Soft drink']
#Soft_drink[['Year','Brand']]

In [4330]:
sodas_ticker = pd.read_csv('..\\Data\\sodas_ticker.txt', sep='-', lineterminator='\r', header=None)
sodas_ticker[0] = sodas_ticker[0].str.split(' ',expand=True)[0].str.replace('\n','')
sodas_ticker = sodas_ticker.rename(columns={0:'row',1:'Ticker'})
sodas_ticker['row'] = pd.to_numeric(sodas_ticker['row'])
Soft_drink = Soft_drink.merge(sodas_ticker, how='left', left_index=True, right_on='row')
Soft_drink = Soft_drink.set_index("row")
Soft_drink.head()

,Year,Product_type,Brand_title,Plot,Date,Brand,Title,Ticker
row,,,,,,,,
4,1979,Soft drink,"Coca-Cola""Hey Kid, Catch!""","A young kid tries to talk to ""Mean"" Joe Greene...",1979-01-21 00:00:00+00:00,Coca-Cola,"Hey Kid, Catch!",KO
16,1992,Soft drink,"Pepsi""Cindy Crawford""",Two Boys see Cindy Crawford pulling up at a ve...,1992-01-26 00:00:00+00:00,Pepsi,Cindy Crawford,PEP
36,1997,Soft drink,"Pepsi""Darth Vader""","While watching Star Wars, a little kid is suck...",1997-01-26 00:00:00+00:00,Pepsi,Darth Vader,PEP
55,1998,Soft drink,"Pepsi""Mosquito""",A CGI fly sucks on a drop of Pepsi. He then gr...,1998-01-25 00:00:00+00:00,Pepsi,Mosquito,PEP
56,1998,Soft drink,"Pepsi""Goose""","A man does aerial tricks with a goose, and the...",1998-01-25 00:00:00+00:00,Pepsi,Goose,PEP


## Next `Wireless`

In [4331]:
wireless = df4[df4.Product_type == 'Wireless']
#wireless[['Year','Brand']]

In [4332]:
wireless_ticker = pd.read_csv('..\\Data\\wireless_ticker.txt', sep='-', lineterminator='\r', header=None)
wireless_ticker[0] = wireless_ticker[0].str.split(' ',expand=True)[0].str.replace('\n','')
wireless_ticker = wireless_ticker.rename(columns={0:'row',1:'Ticker'})
wireless_ticker['row'] = pd.to_numeric(wireless_ticker['row'])
wireless = wireless.merge(wireless_ticker, how='left', left_index=True, right_on='row')
wireless = wireless.set_index("row")
wireless.head()



,Year,Product_type,Brand_title,Plot,Date,Brand,Title,Ticker
row,,,,,,,,
247,2010,Wireless,"Boost Mobile""Shuffle""",Celebrating the 25th anniversary of The Super ...,2010-02-07 00:00:00+00:00,Boost Mobile,Shuffle,S
350,2015,Wireless,"Sprint """"""Super Apology""""""",A caption-styled ad with curse words changed w...,2015-02-01 00:00:00+00:00,Sprint,,SFTBY
351,2015,Wireless,"T-Mobile """"""#KimsDataStash""""""",Kim Kardashian asks viewers to help save the d...,2015-02-01 00:00:00+00:00,T-Mobile,,DTEGY
352,2015,Wireless,"T-Mobile """"""One Upped""""""",Chelsea Handler and Sarah Silverman try to one...,2015-02-01 00:00:00+00:00,T-Mobile,,DTEGY
386,2016,Wireless,"T-Mobile """"""Drop The Balls""""""",In reference to his incorrect announcement of ...,2016-02-07 00:00:00+00:00,T-Mobile,,DTEGY


## Next `Technology`

In [4333]:
technology = df4[df4.Product_type == 'Technology']
#technology[['Year','Brand']]

In [4334]:
tech_ticker = pd.read_csv('..\\Data\\tech_ticker.txt', sep='-', lineterminator='\r', header=None)
tech_ticker[0] = tech_ticker[0].str.split(' ',expand=True)[0].str.replace('\n','')
tech_ticker = tech_ticker.rename(columns={0:'row',1:'Ticker'})
tech_ticker['row'] = pd.to_numeric(tech_ticker['row'])
technology = technology.merge(tech_ticker, how='left', left_index=True, right_on='row')
technology = technology.set_index("row")
technology.head()



,Year,Product_type,Brand_title,Plot,Date,Brand,Title,Ticker
row,,,,,,,,
3,1976,Technology,"Xerox""Monks""",A monk must make 500 handwritten scriptures fo...,1976-01-18 00:00:00+00:00,Xerox,Monks,XRX
5,1981,Technology,"Polaroid""Now with a Flash""",A couple walks in a park. The man shows that n...,1981-01-25 00:00:00+00:00,Polaroid,Now with a Flash,PRD
6,1982,Technology,"RadioShack""Color Computer""",A commercial for the TRS-80 computer.,1982-01-24 00:00:00+00:00,RadioShack,Color Computer,RSHCQ
8,1984,Technology,"Apple Inc.""1984""",In what some call the greatest Super Bowl comm...,1984-01-22 00:00:00+00:00,Apple Inc.,1984,AAPL
9,1984,Technology,"Atari""Get Started""",NaN,1984-01-22 00:00:00+00:00,Atari,Get Started,none


## Now `Clothing`

In [4335]:
clothing = df4[df4.Product_type == 'Clothing']
#clothing[['Year','Brand']]

In [4336]:
clothing_ticker = pd.read_csv('..\\Data\\clothing_ticker.txt', sep='-', lineterminator='\r', header=None)
clothing_ticker[0] = clothing_ticker[0].str.split(' ',expand=True)[0].str.replace('\n','')
clothing_ticker = clothing_ticker.rename(columns={0:'row',1:'Ticker'})
clothing_ticker['row'] = pd.to_numeric(clothing_ticker['row'])
clothing = clothing.merge(clothing_ticker, how='left', left_index=True, right_on='row')
clothing = clothing.set_index("row")
clothing.head()



,Year,Product_type,Brand_title,Plot,Date,Brand,Title,Ticker
row,,,,,,,,
18,1993,Clothing,"Nike""Hare Jordan""",Marvin the Martian takes all the Air Jordans o...,1993-01-31 00:00:00+00:00,Nike,Hare Jordan,NKE
19,1993,Clothing,"Lee""Inhale""",A man trying to suck in his stomach to button ...,1993-01-31 00:00:00+00:00,Lee,Inhale,VFC
20,1993,Clothing,"Lee""The Date""",A young woman in her bedroom upstairs wearing ...,1993-01-31 00:00:00+00:00,Lee,The Date,VFC
26,1994,Clothing,"No Fear""""""Rodeo""""""",The commercial features nine-time World Champi...,1994-01-30 00:00:00+00:00,No Fear,,none
54,1998,Clothing,"Hilfiger""Have Some Fun""",A fancy woman agent is representing Michael Ri...,1998-01-25 00:00:00+00:00,Hilfiger,Have Some Fun,PVH


In [4337]:
sports = df4[df4.Product_type == 'Sports']
#sports[['Year','Brand']]

In [4338]:
sports_ticker = pd.read_csv('..\\Data\\sports_ticker.txt', sep='-', lineterminator='\r', header=None)
sports_ticker[0] = sports_ticker[0].str.split(' ',expand=True)[0].str.replace('\n','')
sports_ticker = sports_ticker.rename(columns={0:'row',1:'Ticker'})
sports_ticker['row'] = pd.to_numeric(sports_ticker['row'])
sports = sports.merge(sports_ticker, how='left', left_index=True, right_on='row')
sports = sports.set_index("row")
sports.head()



,Year,Product_type,Brand_title,Plot,Date,Brand,Title,Ticker
row,,,,,,,,
57,1998,Sports,"NFL""Anthony""",A kid named Anthony is said to play in the NFL.,1998-01-25 00:00:00+00:00,NFL,Anthony,none
79,1999,Sports,"WWF""HQ""","Famous wrestlers, including Stone Cold Steve A...",1999-01-31 00:00:00+00:00,WWF,HQ,none
390,2016,Sports,"NFL """"""Super Bowl Babies Choir""""""","A group of ""Super Bowl Babies"", who were all b...",2016-02-07 00:00:00+00:00,NFL,,none
475,2018,Sports,"2018 Winter Olympics (NBC) """"""Best of U.S.: Li...",Skier Lindsey Vonn prepares for the 2018 Winte...,2018-02-04 00:00:00+00:00,2018 Winter Olympics (NBC),,CMCSA
476,2018,Sports,2018 Winter Olympics (NBC) N/A A number of pro...,NaN,2018-02-04 00:00:00+00:00,2018 Winter Olympics (NBC) N/A A number of pro...,None,CMCSA


## Now  `Gaming`

In [4339]:
gaming = df4[df4.Product_type == 'Gaming']
#gaming[['Year','Brand']]

In [4340]:
gaming_ticker = pd.read_csv('..\\Data\\gaming_ticker.txt', sep='-', lineterminator='\r', header=None)
gaming_ticker[0] = gaming_ticker[0].str.split(' ',expand=True)[0].str.replace('\n','')
gaming_ticker = gaming_ticker.rename(columns={0:'row',1:'Ticker'})
gaming_ticker['row'] = pd.to_numeric(gaming_ticker['row'])
gaming = gaming.merge(gaming_ticker, how='left', left_index=True, right_on='row')
gaming = gaming.set_index("row")
gaming.head()



,Year,Product_type,Brand_title,Plot,Date,Brand,Title,Ticker
row,,,,,,,,
127,2001,Gaming,"PlayStation 2""Future""",A future commercial advertises the PS9.,2001-01-28 00:00:00+00:00,PlayStation 2,Future,SNE
138,2002,Gaming,Madden NFL 2002,TV promo.,2002-02-03 00:00:00+00:00,Madden NFL 2002,None,EA
155,2004,Gaming,"PlayStation 2""Going Back""",TV promo for Jet Li: Rise to Honor (Full Versi...,2004-02-01 00:00:00+00:00,PlayStation 2,Going Back,SNE
343,2015,Gaming,"Clash of Clans """"""Revenge""""""",Liam Neeson plans to avenge his loss in the ga...,2015-02-01 00:00:00+00:00,Clash of Clans,,none
344,2015,Gaming,"Game of War: Fire Age """"""Who I Am""""""",Kate Upton's character leads her troops agains...,2015-02-01 00:00:00+00:00,Game of War: Fire Age,,none


In [4341]:
lexus_blackpanther = df4[df4.Product_type == 'Car / Film']

## Now merge them back together.

In [4342]:
tickers_added = pd.concat([films, tvshows, cars, alcohol, food, website, goods, services, Soft_drink, wireless, clothing, technology, sports,gaming ])
tickers_added = tickers_added.reset_index()
tickers_added.head(30)

,row,Year,Product_type,Brand_title,Plot,Date,Brand,Title,Ticker
0,27,1996,Film,Independence Day,TV promo.,1996-01-28 00:00:00+00:00,Independence Day,None,FOX
1,30,1997,Film,Dante's Peak,TV promo.,1997-01-26 00:00:00+00:00,Dante's Peak,None,CMCSA
2,31,1997,Film,The Lost World: Jurassic Park,TV promo.,1997-01-26 00:00:00+00:00,The Lost World: Jurassic Park,None,CMCSA
3,32,1997,Film,The Saint,TV promo.,1997-01-26 00:00:00+00:00,The Saint,None,VIAC
4,43,1998,Film,Armageddon,TV promo.,1998-01-25 00:00:00+00:00,Armageddon,None,DIS
5,44,1998,Film,Lost in Space,TV promo.,1998-01-25 00:00:00+00:00,Lost in Space,None,N/A
6,45,1998,Film,Mercury Rising,TV promo.,1998-01-25 00:00:00+00:00,Mercury Rising,None,CMCSA
7,46,1998,Film,Sphere,TV promo.,1998-01-25 00:00:00+00:00,Sphere,None,TWX
8,47,1998,Film,The Mask of Zorro,TV promo.,1998-01-25 00:00:00+00:00,The Mask of Zorro,None,SNE
9,48,1998,Film,The Wedding Singer,TV promo.,1998-01-25 00:00:00+00:00,The Wedding Singer,None,N/A


## More cleaning and add `Mkt_Date` and `Mkt_Date_plus6` 
These are chosen to be the Monday after the Superbowl and 183 days following that date.

In [4343]:
# explain these and add cell above with `.unique`
tickers_added.Ticker = tickers_added.Ticker.str.replace(" ", "").str.replace("OTC:", "").str.replace("NYSE:", "")
tickers_added.Ticker = tickers_added.Ticker.str.replace("NASDAQ:", "").str.replace("SpectrumBrandsHoldings,Inc.\(SPB\)", "SPB")
tickers_added.Ticker = tickers_added.Ticker.str.replace("\(Unilever\)", "")
tickers_added['Mkt_Date'] = tickers_added['Date'] + pd.Timedelta(days=1)
tickers_added['Mkt_Date_plus6'] = tickers_added['Date'] + pd.Timedelta(days=183)
tickers_added['Mkt_Date'] = tickers_added['Mkt_Date'].dt.strftime('%m-%d-%Y')
tickers_added['Mkt_Date_plus6'] = tickers_added['Mkt_Date_plus6'].dt.strftime('%m-%d-%Y')
tickers_added.head()

C:\Users\micha\AppData\Local\Temp\ipykernel_13364\1901507258.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  tickers_added.Ticker = tickers_added.Ticker.str.replace("NASDAQ:", "").str.replace("SpectrumBrandsHoldings,Inc.\(SPB\)", "SPB")
C:\Users\micha\AppData\Local\Temp\ipykernel_13364\1901507258.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  tickers_added.Ticker = tickers_added.Ticker.str.replace("\(Unilever\)", "")


,row,Year,Product_type,Brand_title,Plot,Date,Brand,Title,Ticker,Mkt_Date,Mkt_Date_plus6
0,27,1996,Film,Independence Day,TV promo.,1996-01-28 00:00:00+00:00,Independence Day,None,FOX,01-29-1996,07-29-1996
1,30,1997,Film,Dante's Peak,TV promo.,1997-01-26 00:00:00+00:00,Dante's Peak,None,CMCSA,01-27-1997,07-28-1997
2,31,1997,Film,The Lost World: Jurassic Park,TV promo.,1997-01-26 00:00:00+00:00,The Lost World: Jurassic Park,None,CMCSA,01-27-1997,07-28-1997
3,32,1997,Film,The Saint,TV promo.,1997-01-26 00:00:00+00:00,The Saint,None,VIAC,01-27-1997,07-28-1997
4,43,1998,Film,Armageddon,TV promo.,1998-01-25 00:00:00+00:00,Armageddon,None,DIS,01-26-1998,07-27-1998


In [4344]:
notkr = ['N/A', 'Private(Mars,Inc.)', 'Private', 'Private(MexicanHassAvocadoImportersAssociation)', "Private(Arby'sRestaurantGroup,Inc.)" , 'none', 'NONE',':none']  
no_ticker = tickers_added[tickers_added["Ticker"].isin(notkr)]
cln_tickers = tickers_added.drop(tickers_added[tickers_added["Ticker"].isin(notkr)].index)
cln_tickers.Ticker.unique()

array(['FOX', 'CMCSA', 'VIAC', 'DIS', 'TWX', 'SNE', 'GE', 'MGM', 'FOXA',
       'VIA', 'NWSA', 'DWA', 'LGF.A', 'OXYGN', 'CBS', 'NFLX', 'AMZN', 'T',
       'FCAU', 'GM', 'NSANY', 'FUJHY', 'HMC', 'F', 'VWAGY', 'HYMTF',
       'BMWYY', 'TM', 'DAI', 'POAHY', 'MIL', 'BUD', 'HEINY', 'SAB',
       'LVMUY', 'CEECF', 'MCD', 'PEP', 'K', 'HRL', 'YUM', 'SPB', 'UL',
       'DENN', 'LKQ', 'KHC', 'JACK', 'WEN', 'QSR', 'ABTL', 'BUYX', 'HOTJ',
       'MWW', 'AUTO', 'EDS', 'ETFC', 'STMP', 'KFRC', 'LMIN', 'LFMN',
       'MSTR', 'OURN', 'IPET', 'WBMD', 'YHOO', 'CARS', 'GDDY', 'GOOGL',
       'WIX', 'ABNB', 'FB', 'FBHS', 'MGDDF', 'BRDCY', 'EA', 'BBY',
       'HENKY', 'BMY', 'PG', 'CL', 'NTDOY', 'MNST', 'WMT', 'MSFT', 'AXP',
       'V', 'FDX', 'MA', 'DFS', 'ALL', 'WTW', 'INTU', 'TKHLY', 'RTK',
       'GRPN', 'KO', 'S', 'SFTBY', 'DTEGY', 'VZ', 'NKE', 'VFC', 'PVH',
       'SKX', 'LB', 'NWL', 'XRX', 'PRD', 'RSHCQ', 'AAPL', 'INTC', 'OCPNY'],
      dtype=object)

Add exchange by Chatgpt

In [4345]:
ticker_symbols = [('FOX', 'NASDAQ'), ('CMCSA', 'NASDAQ'),('MCD', 'NYSE'), ('PEP', 'NASDAQ'), ('K', 'NYSE'), ('HRL', 'NYSE'), ('BUD', 'NYSE'),  ('DIS', 'forbes2000'), ('TWX', 'forbes2000') , ('GE', 'NYSE'),  ('NFLX', 'NASDAQ') , ('AMZN', 'NASDAQ'), ('T', 'NYSE'),  ('GM', 'NYSE') , ('HMC', 'NYSE'), ('F', 'NYSE'), ('TM', 'NYSE'),  
('FUJHY', 'OTC'), ('NSANY', 'OTC'), ('VWAGY', 'OTC'), ('HYMTF', 'OTC'), ('BMWYY', 'OTC'),  ('DAI', 'OTC'), 
('POAHY', 'OTC'), ('MIL', 'OTC'), ('HEINY', 'OTC'), ('SAB', 'OTC'), ('LVMUY', 'OTC'), ('CEECF', 'OTC'),
('YUM', 'NYSE'), ('SPB', 'NYSE'), 
('UL', 'NYSE'), ('DENN', 'NASDAQ'), ('LKQ', 'NASDAQ'), ('KHC', 'NASDAQ'), ('JACK', 'NASDAQ'), ('WEN', 'NASDAQ'),
('QSR', 'NYSE'), ('ABTL', 'OTC'), ('BUYX', 'OTC'), ('HOTJ', 'OTC'), ('MWW', 'NYSE'), ('AUTO', 'NASDAQ'), ('EDS', 'OTC'),
('ETFC', 'NASDAQ'), ('STMP', 'NASDAQ'), ('LGF.A', 'NYSE'), ('KFRC', 'OTC'), ('LMIN', 'OTC'), ('LFMN', 'OTC'), 
('MSTR', 'NASDAQ'), ('OURN', 'OTC'), ('IPET', 'OTC'), ('WBMD', 'NASDAQ'), ('YHOO', 'NASDAQ'), ('CARS', 'NASDAQ'), 
('GDDY', 'NYSE'), ('GOOGL', 'NASDAQ'), ('WIX', 'NASDAQ'), ('ABNB', 'NASDAQ'), ('FB', 'NASDAQ'), ('FBHS', 'NYSE'), 
('MGDDF', 'OTC'), ('BRDCY', 'OTC'), ('EA', 'NASDAQ'), ('BBY', 'NYSE'), ('HENKY', 'OTC'), ('BMY', 'NYSE'), ('PG', 'NYSE'), 
('CL', 'NYSE'), ('NTDOY', 'OTC'), ('MNST', 'NASDAQ'), ('WMT', 'NYSE'), ('MSFT', 'NASDAQ'), ('AXP', 'NYSE'), ('V', 'NYSE'),
('FDX', 'NYSE'), ('MA', 'NYSE'), ('DFS', 'NYSE'), ('ALL', 'NYSE'), ('WTW', 'NASDAQ'), ('INTU', 'NASDAQ'), ('TKHLY', 'OTC')
, ('RTK', 'OTC'), ('GRPN', 'NASDAQ'), ('KO', 'NYSE'), ('S', 'NASDAQ'), ('SFTBY', 'OTC'), ('DTEGY', 'OTC'), ('VZ', 'NYSE')
,('SNE', 'NYSE'),('VIA', 'NYSE'), ('MGM', 'NYSE'), ('FOXA', 'NASDAQ'), ('NWSA', 'NASDAQ'), ('DWA', 'NYSE'), 
('OXYGN', 'NASDAQ'), ('CBS', 'NYSE'), ('FCAU', 'NYSE') ]
ticker_symbols = [(t[0],t[1].lower()) for t in ticker_symbols]
ticker_symbols[0] = ('FOXA', 'forbes2000')
tkrs = pd.DataFrame(ticker_symbols, columns=['Ticker', 'xchange'])
tkrs.head()

,Ticker,xchange
0,FOXA,forbes2000
1,CMCSA,nasdaq
2,MCD,nyse
3,PEP,nasdaq
4,K,nyse


The `OTC` stocks are not in the current database so drop them out for now.

## Next want to add the date from the `df3` set

Drop the `otc` stocks for now.

## Next dataset

This is the stock price database from kaggle
https://www.kaggle.com/datasets/paultimothymooney/stock-market-data/code?resource=download

Create a `load_stock` function, then load all the stocks in the list of tickers that are found into `stocks`

In [4346]:
import numpy as np 
import pandas as pd 
import datetime as dt


def load_stock(name):
    '''function to load stock data from the path'''
    file = Path("..\\Data\\stock_market_data\\forbes2000\\csv\\{}.csv".format(name))
    if file.exists():
        df=pd.read_csv(file)
        return df
    else:
        tkrs[tkrs['Ticker'] == name]['xchange'] = 'otc'
        column_names = []
        return pd.DataFrame(column_names, columns=['Date'])
# Add an exception and change tkrs.xchange to'otc' then skip

names = tkrs.Ticker
stocks=[load_stock(n) for n in names]
names.reset_index(drop=True, inplace=True)
names;

C:\Users\micha\AppData\Local\Temp\ipykernel_13364\3588370420.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tkrs[tkrs['Ticker'] == name]['xchange'] = 'otc'
C:\Users\micha\AppData\Local\Temp\ipykernel_13364\3588370420.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tkrs[tkrs['Ticker'] == name]['xchange'] = 'otc'
C:\Users\micha\AppData\Local\Temp\ipykernel_13364\3588370420.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

C:\Users\micha\AppData\Local\Temp\ipykernel_13364\3588370420.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tkrs[tkrs['Ticker'] == name]['xchange'] = 'otc'
C:\Users\micha\AppData\Local\Temp\ipykernel_13364\3588370420.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tkrs[tkrs['Ticker'] == name]['xchange'] = 'otc'
C:\Users\micha\AppData\Local\Temp\ipykernel_13364\3588370420.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

Merge the `tickers_added` dataframe with the stock information for the `Mkt_Date` and the `Mkt_Date_plus6` for each stock in  `stocks` creating a list of dataframes which are concatenated into `df_test` 


In [4347]:
dfs =[]
for i,stock in enumerate(stocks):
    stock1 = stock
    stock1['Ticker'] = names[i]
    #print(stock.Date.min())
    stock1['Date'] = pd.to_datetime(stock1['Date'] , format="%d-%m-%Y", utc=True)
    stock1['Date'] = stock1['Date'].dt.strftime("%m-%d-%Y")
    #stock1 = stock1.drop(columns=['Low','Open','High','Volume', 'Adjusted Close'])
    #print(stock1.info())
    #print(stock1.head())
    df_1 = tickers_added.merge(stock1, how='left', left_on=['Ticker','Mkt_Date'], right_on=['Ticker','Date'])
    df_2 = df_1.merge(stock1, how='left', left_on=['Ticker', 'Mkt_Date_plus6'],  right_on=['Ticker','Date'])
    df_2 = df_2[(df_2.Mkt_Date == df_2.Date_y) | (df_2.Mkt_Date_plus6 == df_2.Date)]
    #print(df_1.info())
    dfs.append(df_2)
    #dataframe1 = dataframe1.merge(stock, on='Date')
    #print(dataframe1['Date'].head())
    #print(stock.loc[stock['Date']=='2019-03-12'])
    #dataframe1 = pd.merge(dataframe1, stock, how='outer', on='Date')
    #print(df_new.head())
    #for t in df_1.Date:
        #stock.Close[stock['Date']== t]
df_test = pd.concat(dfs) 
#tickers_added.head()

In [4348]:
print(df_test.shape)
df_test.info()

(310, 25)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 310 entries, 1 to 51
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   row               310 non-null    int64              
 1   Year              310 non-null    object             
 2   Product_type      310 non-null    object             
 3   Brand_title       310 non-null    object             
 4   Plot              300 non-null    object             
 5   Date_x            310 non-null    datetime64[ns, UTC]
 6   Brand             310 non-null    object             
 7   Title             206 non-null    object             
 8   Ticker            310 non-null    object             
 9   Mkt_Date          310 non-null    object             
 10  Mkt_Date_plus6    310 non-null    object             
 11  Date_y            305 non-null    object             
 12  Low_x             305 non-null    float64            
 

In [4349]:
# Drop some columns Low_x ,Open_x,  Volume_x,  High_x, Adjusted Close_x, Date, Low_y, Open_y, Volume_y  High_y, Adjusted Close_y                     
df_test = df_test.drop(columns=['Low_x','Open_x','Volume_x', 'High_x', 'Adjusted Close_x', 'Low_y', 'Open_y', 'Volume_y' , 'High_y', 'Adjusted Close_y']) 
#df_test[['row','Ticker','Mkt_Date','Date_y','Close_x','Mkt_Date_plus6','Date' ,'Close_y']].tail(20)
# Rename columns to match df_excel
df_test = df_test.rename(columns={'Close_x':'close','Close_y':'close_plus6', 'Date_y':'Date_Mkt', 'Date':'Date_MktPlus'})
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 310 entries, 1 to 51
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   row             310 non-null    int64              
 1   Year            310 non-null    object             
 2   Product_type    310 non-null    object             
 3   Brand_title     310 non-null    object             
 4   Plot            300 non-null    object             
 5   Date_x          310 non-null    datetime64[ns, UTC]
 6   Brand           310 non-null    object             
 7   Title           206 non-null    object             
 8   Ticker          310 non-null    object             
 9   Mkt_Date        310 non-null    object             
 10  Mkt_Date_plus6  310 non-null    object             
 11  Date_Mkt        305 non-null    object             
 12  close           305 non-null    float64            
 13  Date_MktPlus    310 non-null    obje

Leave this dataset for now

## Now that the tickers are in place export the file to excel

With this file in excel use the `stocks` datatype, convert the dates in excel and extract closing prices from `STOCKHISTORY` and from the general data for the stock extract `year_inc`, `market_cap_cur` and `shares_out` for the year of incorporation, current market cap, and current shares outstanding. 

In [4350]:
excel = tickers_added[['Year','Product_type','Brand','Title','Ticker','Mkt_Date','Mkt_Date_plus6']]
excel.to_excel('output.xlsx')

In [4351]:
#May want to redo this as something seems to happen to the tickers
#df_excel = pd.read_excel('output1.xlsx')
df_excel = pd.read_excel('output2.xlsx')
df_excel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556 entries, 0 to 555
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   row             556 non-null    int64         
 1   Year            556 non-null    int64         
 2   Product_type    556 non-null    object        
 3   Brand           556 non-null    object        
 4   Title           189 non-null    object        
 5   Ticker          541 non-null    object        
 6   Mkt_Date        556 non-null    datetime64[ns]
 7   Mkt_Date_plus6  556 non-null    datetime64[ns]
 8   close           346 non-null    object        
 9   close_plus6     346 non-null    object        
 10  market_cap_cur  425 non-null    float64       
 11  shares_out      448 non-null    float64       
 12  year_inc        398 non-null    float64       
 13  spx_close       541 non-null    float64       
 14  spx_plus6       547 non-null    float64       
 15  rtn_vs

Look at missing `spx` and `rtn` 

In [4352]:
#look at missing SPX values
df_excel[df_excel['spx'].isnull()]

,row,Year,Product_type,Brand,Title,Ticker,Mkt_Date,Mkt_Date_plus6,close,close_plus6,market_cap_cur,shares_out,year_inc,spx_close,spx_plus6,rtn_vs_spx,rtn,spx
163,0,1969,Car,Chrysler,Scuba Diver,FCAU,1969-01-13,1969-07-14,NaN,NaN,4.820181e+10,3.213454e+09,2014.0,NaN,NaN,NaN,NaN,NaN
164,1,1970,Car,Pontiac,Humbler,GM,1970-01-12,1970-07-13,NaN,NaN,4.572813e+10,1.390124e+09,2009.0,NaN,NaN,NaN,NaN,NaN
165,14,1990,Car,Nissan,Turbocharged Dream,NSANY,1990-01-29,1990-07-30,18.88,13.88,1.558146e+10,2.110358e+09,1933.0,NaN,264.92,NaN,-0.264901,NaN
244,10,1984,Alcohol,Miller Lite,Train,MIL,1984-01-23,1984-07-23,NaN,NaN,5.980666e+09,1.213117e+09,2001.0,NaN,187.38,NaN,NaN,NaN
245,13,1989,Alcohol,Budweiser,NaN,BUD,1989-01-23,1989-07-24,NaN,NaN,1.132130e+11,1.737197e+09,NaN,221.85,NaN,NaN,NaN,NaN
304,7,1982,Food,McDonald's,Hopscotch,MCD,1982-01-25,1982-07-26,1.47,1.85,2.174144e+11,7.300317e+08,1964.0,NaN,NaN,NaN,0.257896,NaN
305,12,1988,Food,Doritos tortilla chips,NaN,PEP,1988-02-01,1988-08-01,5.48,6.04,2.631600e+11,1.377693e+09,1986.0,NaN,201.38,NaN,0.102665,NaN
407,2,1975,Goods,Master Lock,Rifle,FBHS,1975-01-13,1975-07-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
408,11,1984,Goods,Master Lock,The Doubters,FBHS,1984-01-23,1984-07-23,NaN,NaN,6.165164e+11,2.562732e+09,2004.0,NaN,187.38,NaN,NaN,NaN
440,15,1990,Service,American Express,The Race,AXP,1990-01-29,1990-07-30,8.79,8.54,1.135895e+11,7.432407e+08,1965.0,NaN,264.92,NaN,-0.029164,NaN


The values missing `spx` are missing at least one close of the S&P and are essentially the old ads which are the first 15 original rows. The Data was sparse for the years anyway so dropping thes out is probably a good idea anyway.

In [4353]:
df_excel[~((df_excel['spx'].isnull())|(df_excel['rtn'].isnull()))].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 333 entries, 1 to 553
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   row             333 non-null    int64         
 1   Year            333 non-null    int64         
 2   Product_type    333 non-null    object        
 3   Brand           333 non-null    object        
 4   Title           87 non-null     object        
 5   Ticker          333 non-null    object        
 6   Mkt_Date        333 non-null    datetime64[ns]
 7   Mkt_Date_plus6  333 non-null    datetime64[ns]
 8   close           333 non-null    object        
 9   close_plus6     333 non-null    object        
 10  market_cap_cur  327 non-null    float64       
 11  shares_out      328 non-null    float64       
 12  year_inc        301 non-null    float64       
 13  spx_close       333 non-null    float64       
 14  spx_plus6       333 non-null    float64       
 15  rtn_vs

### Not bad
333 prices out of 556 rows. 
Now just want to see if we caught some in our earlier attempt that were not captured here.

See what happens if we merge on rows

In [4354]:
df_try = df_excel.merge(df_test, how='left', on=['row','Ticker','Brand','Product_type','Title'])
df_try.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 556 entries, 0 to 555
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   row               556 non-null    int64              
 1   Year_x            556 non-null    int64              
 2   Product_type      556 non-null    object             
 3   Brand             556 non-null    object             
 4   Title             189 non-null    object             
 5   Ticker            541 non-null    object             
 6   Mkt_Date_x        556 non-null    datetime64[ns]     
 7   Mkt_Date_plus6_x  556 non-null    datetime64[ns]     
 8   close_x           346 non-null    object             
 9   close_plus6_x     346 non-null    object             
 10  market_cap_cur    425 non-null    float64            
 11  shares_out        448 non-null    float64            
 12  year_inc          398 non-null    float64            
 13  spx_c

Fill any blanks for the closing prices in the left dataframe `df_excel` with values in the right `df_test`

Then check how many were filled in by checking the shape on the intersection.

In [4359]:
df_try['close_x'] = df_try['close_x'].fillna(df_try['close_y'])
df_try['close_plus6_x'] = df_try['close_plus6_x'].fillna(df_try['close_plus6_y'])
print(df_try[df_try.close_x == df_try.close_y].shape)
print(df_try[df_try.close_plus6_x == df_try.close_plus6_y].shape)
df_try.info()

(30, 28)
(33, 28)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 556 entries, 0 to 555
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   row               556 non-null    int64              
 1   Year_x            556 non-null    int64              
 2   Product_type      556 non-null    object             
 3   Brand             556 non-null    object             
 4   Title             189 non-null    object             
 5   Ticker            541 non-null    object             
 6   Mkt_Date_x        556 non-null    datetime64[ns]     
 7   Mkt_Date_plus6_x  556 non-null    datetime64[ns]     
 8   close_x           368 non-null    object             
 9   close_plus6_x     369 non-null    object             
 10  market_cap_cur    425 non-null    float64            
 11  shares_out        448 non-null    float64            
 12  year_inc          398 non-null    float64     

In [4356]:
print(tickers_added.Ticker.unique())
print(df_test.Ticker.unique())

['FOX' 'CMCSA' 'VIAC' 'DIS' 'N/A' 'TWX' 'SNE' 'GE' 'MGM' 'FOXA' 'VIA'
 'NWSA' 'DWA' 'LGF.A' 'OXYGN' 'CBS' 'NFLX' 'AMZN' 'T' 'FCAU' 'GM' 'NSANY'
 'FUJHY' 'HMC' 'F' 'VWAGY' 'HYMTF' 'BMWYY' 'TM' 'DAI' 'POAHY' 'MIL' 'BUD'
 'HEINY' 'SAB' 'LVMUY' 'CEECF' 'MCD' 'PEP' 'K' 'HRL' 'Private' 'YUM'
 'Private(Mars,Inc.)' 'SPB' 'UL' 'DENN' 'LKQ'
 'Private(MexicanHassAvocadoImportersAssociation)' 'KHC'
 "Private(Arby'sRestaurantGroup,Inc.)" 'JACK' 'WEN' 'QSR' 'ABTL' 'BUYX'
 'HOTJ' 'MWW' 'AUTO' 'none' 'NONE' 'EDS' 'ETFC' 'STMP' 'KFRC' 'LMIN'
 'LFMN' 'MSTR' 'OURN' 'IPET' 'WBMD' 'YHOO' 'CARS' 'GDDY' 'GOOGL' 'WIX'
 'ABNB' 'FB' 'FBHS' 'MGDDF' 'BRDCY' 'EA' 'BBY' ':none' 'HENKY' 'BMY' 'PG'
 'CL' 'NTDOY' 'MNST' 'WMT' 'MSFT' 'AXP' 'V' 'FDX' 'MA' 'DFS' 'ALL' 'WTW'
 'INTU' 'TKHLY' 'RTK' 'GRPN' 'KO' 'S' 'SFTBY' 'DTEGY' 'VZ' 'NKE' 'VFC'
 'PVH' 'SKX' 'LB' 'NWL' 'XRX' 'PRD' 'RSHCQ' 'AAPL' 'INTC' 'OCPNY']
['CMCSA' 'MCD' 'PEP' 'K' 'HRL' 'BUD' 'DIS' 'TWX' 'GE' 'NFLX' 'AMZN' 'T'
 'GM' 'HMC' 'F' 'TM' 'NSANY' 'HYMTF' 'POA

In [4357]:
df5 = pd.read_csv('..\\Data\\advertisers.txt', sep='\t', lineterminator='\r')
# Text file scraped from http://superadsdb.com/advertisers
df5.head(20)

,Company\Brand,Ads,Industry,Latest,First,Links
0,\n21st Century Studios,14,Movies,2019,1994,NaN
1,\n7 Up,11,Soft Drinks,2001,1992,NaN
2,\n Diet 7 Up,3,Soft Drinks,1992,--,NaN
3,\n84 Lumber,1,Building Supplies,2017,--,NaN
4,\nAccenture,4,Technology,2001,--,NaN
5,\nAcura,3,Automobiles,2016,2001,NaN
6,\n Acura MDX,1,SUVs,2001,--,NaN
7,\n Acura NSX,2,Sports Cars,2016,2012,NaN
8,\nADT,2,Home Security,2020,2019,NaN
9,\nAdvil,5,Pain Relief,2016,1985,NaN
